<a href="https://colab.research.google.com/github/JeyTee1988/LLM-Schulung/blob/main/llm_schulung.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Wichtig**: Wenn Du dieses Notebook in Google Colab ausführst, wähle zuerst unter Laufzeit > Laufzeittyp ändern eine GPU aus

Zunächst installieren wir die benötigte Library Huggingface Transformer

In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 43.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 74.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 23.8 MB/s eta 0:00:00


# Klassifizierung von Text

Wir laden eine Pipeline mit einem vortrainierten Sentiment-Modell. Das Modell kann explizit angegeben werden, ansonsten wird ein Standard-Modell genommen.

In [2]:
from transformers import pipeline
classifier = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


Wir können das Modell nutzen, um die Stimmung eines Textes zu bestimmen. Hierbei wird angegeben, ob der Text positiv oder negativ ist und wie sicher das Modell ist bei der Bewertung.

In [3]:
classifier(["We are happy to have you participate in our modest LLM workshop!", "Sorry that we had to cap the participant limit to 20"])

[{'label': 'POSITIVE', 'score': 0.9998558759689331},
 {'label': 'NEGATIVE', 'score': 0.9991243481636047}]

Dieses Modell wurde nur mit englischen Texten und scheinbar nicht mit Emoji trainiert:

In [4]:
classifier(["Was ein schöner Tag für eine Schulung", "🤗", "😠"])

[{'label': 'NEGATIVE', 'score': 0.9137823581695557},
 {'label': 'NEGATIVE', 'score': 0.6970564126968384},
 {'label': 'NEGATIVE', 'score': 0.6970564126968384}]

# Textgenerierung

Wir laden das vortrainierte distilgpt2 Modell und nutzen es für Textgenerierung

In [5]:
from transformers import pipeline

pipe = pipeline("text-generation", model="distilgpt2")

Wir können das Modell nutzen, um Texte zu vervollständigen.

In [6]:
from transformers import set_seed
set_seed(42)
pipe("Hello, I’m a language model", max_length=20, num_return_sequences=5)



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Hello, I’m a language model but what I do in that role is to make everything'},
 {'generated_text': 'Hello, I’m a language model.” You’ll know the real name'},
 {'generated_text': 'Hello, I’m a language model’s\xadself, and I’m'},
 {'generated_text': 'Hello, I’m a language model, and I really wanted to make a nice API,'},
 {'generated_text': 'Hello, I’m a language model. I’think in any language. I�'}]

# Tokenizer

Ein Tokenizer in einem Language Learning Model (LLM) dient dazu, Text in kleinere Einheiten, sogenannte Tokens, zu zerlegen. Diese Tokens können einzelne Wörter, Phrasen oder sogar einzelne Buchstaben sein.

Der Tokenizer ist wichtig, weil ein LLM wie ein neuronales Netzwerk arbeitet, das auf Zahlen, nicht auf Wörtern, basiert. Es kann nicht direkt Text verstehen oder interpretieren. Stattdessen muss es den Text in eine numerische Form umwandeln, die es verarbeiten kann.

Dieser Prozess des Zerlegens und Umwandeln wird durch den Tokenizer durchgeführt. Er nimmt den ursprünglichen Text, teilt ihn in Tokens und wandelt diese dann in Zahlen um, die das Modell verarbeiten kann.

https://platform.openai.com/tokenizer


In [7]:
tokenizer = pipe.tokenizer
model = pipe.model

text = "Hello, I’m a language model"
tokenizer.tokenize(text)

['Hello', ',', 'ĠI', 'âĢ', 'Ļ', 'm', 'Ġa', 'Ġlanguage', 'Ġmodel']

In [8]:
encoded_input = tokenizer(text, return_tensors='pt')
encoded_input.input_ids

tensor([[15496,    11,   314,   447,   247,    76,   257,  3303,  2746]])

Der Tokenizer enthält ein Dictionary, welches jedem Token eine Zahl zuweist

In [9]:
vocab = tokenizer.get_vocab()
first_20_items = {k: vocab[k] for k in list(vocab.keys())[:20]}
first_20_items

{'osuke': 45914,
 'Ġwatered': 48024,
 'ĠChristianity': 13624,
 'header': 25677,
 'ches': 2052,
 'ĠLak': 30392,
 'Fill': 33762,
 'Ġepigen': 49651,
 'Ġcondu': 36118,
 '8': 23,
 'ĠMedic': 5786,
 'Ġobserve': 12414,
 'Ġrumours': 30427,
 'Ġnumeric': 35575,
 'Ġwraps': 27521,
 'ĠAki': 48663,
 'Ġcrossover': 27668,
 'ĠSmoke': 25416,
 'hound': 39047,
 'eday': 23712}

Es gibt insgesamt 50257 Tokens in diesem Tokenizer.

In [10]:
len(vocab)

50257

# Die Ausgabe des Transformers verstehen

Wir können auch manuell die Ausgabe des Encoders durch unser Modell schleusen, um zu sehen, was es damit macht.

Obwohl wir nur den fehlenden Text vervollständigen wollen, liefert unser Transformer tatsächlich für **jedes** Token in unserem Input-Text eine Vorhersage für das nächste Token:

In [11]:
output = model(**encoded_input)
token_logits = output.logits

# Get the predicted token ids
predicted_token_ids = token_logits.argmax(dim=-1)
predicted_token_ids

tensor([[ 383,  290,  447,  247,   76,  407, 4336, 5887,   13]])

In [12]:
# Decode the tokens to text
predicted_text = [tokenizer.decode(t) for t in predicted_token_ids]

for index, t in enumerate(predicted_token_ids[0]):
  input_token = tokenizer.tokenize(text)[index]
  predicted_next_token = tokenizer.decode(t)
  print(input_token + " => " + predicted_next_token)


Hello =>  The
, =>  and
ĠI => �
âĢ => �
Ļ => m
m =>  not
Ġa =>  fan
Ġlanguage =>  expert
Ġmodel => .


Um eine größere Spanne von Texten erzeugen zu können, nutzen Transformer nicht immer die wahrscheinlichste Vorhersage (Stichwort **Temperatur**).

Hier sind die fünf wahrscheinlichsten Vorhersagen:

In [17]:
import torch

# Convert logits to probabilities
token_probs = torch.nn.functional.softmax(token_logits, dim=-1)

# Get the top 5 predicted token ids
top_k = 5
top_k_probs, top_k_indices = token_probs.topk(top_k, dim=-1)

# Iterate over the tokens and print the top 5 predictions for each one
for i in range(top_k_indices.shape[1]):
    input_token = tokenizer.tokenize(text)[i]
    print(f"Input: {input_token}:")
    for j in range(top_k):
        token_id = top_k_indices[0, i, j].item()
        probability = top_k_probs[0, i, j].item()
        output_token = tokenizer.decode([token_id])
        print(f"    Output: {output_token}, probability: {probability:.3%}")


Input: Hello:
    Output:  The, probability: 4.253%
    Output: ., probability: 1.890%
    Output: ,, probability: 1.780%
    Output:  A, probability: 1.263%
    Output: The, probability: 1.190%
Input: ,:
    Output:  and, probability: 8.286%
    Output:  the, probability: 4.592%
    Output:  we, probability: 4.250%
    Output:  I, probability: 3.777%
    Output:  you, probability: 3.224%
Input: ĠI:
    Output: �, probability: 16.135%
    Output:  am, probability: 7.065%
    Output: 'm, probability: 6.285%
    Output:  have, probability: 4.643%
    Output:  think, probability: 4.031%
Input: âĢ:
    Output: �, probability: 39.789%
    Output: �, probability: 11.001%
    Output: �, probability: 7.779%
    Output: �, probability: 4.514%
    Output: �, probability: 4.339%
Input: Ļ:
    Output: m, probability: 53.070%
    Output: ve, probability: 13.964%
    Output: ll, probability: 8.219%
    Output: d, probability: 3.578%
    Output: �, probability: 1.601%
Input: m:
    Output:  not, prob

# Weitere Arten von Tasks
Auf folgender Seite findet ihr weitere mögliche Anwendungsfälle
https://huggingface.co/docs/transformers/main/en/quicktour#pipeline

Probiert gerne welche aus 🤗